In [14]:
import nibabel as nib
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from statistics import mean
from skimage.transform import resize
import glob
import PIL
import copy
import cv2
import csv
from tqdm import tqdm
from skimage import io, color, transform
import random
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow import keras
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [4]:
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install tensorflow 

In [ ]:
#@title Visualizing information about one image
img = nib.load('/content/drive/MyDrive/og_pazienti/case_00074/imaging_middle.nii.gz')

print(img.shape)
print(img.get_data_dtype())
print(img.header)

# Image Resizing

In [ ]:
# Middle slice picking

import os
import nibabel as nib
import numpy as np

og_path = '/content/drive/MyDrive/og_pazienti'

def extract_middle_slice(img_path, mask_path):
    img = nib.load(img_path).get_fdata()
    mask = nib.load(mask_path).get_fdata()

    # Get the middle slice index of the first dimension
    slice_index = img.shape[0] // 2

    # Extract the middle slice from the image and segmentation
    img_middle_slice = img[slice_index, :, :]
    mask_middle_slice = mask[slice_index, :, :]

    return img_middle_slice, mask_middle_slice

def process_subfolders(og_path):
    for case_folder in os.listdir(og_path):
        case_path = os.path.join(og_path, case_folder)
        img_path = os.path.join(case_path, "imaging.nii.gz")
        mask_path = os.path.join(case_path, "segmentation.nii.gz")

        if not os.path.isfile(img_path) or not os.path.isfile(mask_path):
            continue  # Skip if the image or mask file is missing

        img_middle_slice, mask_middle_slice = extract_middle_slice(img_path, mask_path)

        # Save the middle slice image to the original subfolder
        img_out_path = os.path.join(case_path, "imaging_middle.nii.gz")
        nib.save(nib.Nifti1Image(img_middle_slice, affine=None), img_out_path)

        # Save the middle slice segmentation to the original subfolder
        mask_out_path = os.path.join(case_path, "segmentation_middle.nii.gz")
        nib.save(nib.Nifti1Image(mask_middle_slice, affine=None), mask_out_path)

        # Delete the old image and segmentation files
        os.remove(img_path)
        os.remove(mask_path)

# Call the function to process each subfolder
process_subfolders(og_path)

In [8]:
# Resizing

SIZE_X = 256
SIZE_Y = 256

main_folder = '/content/drive/MyDrive/og_pazienti'
out_img_dir = '/content/drive/MyDrive/resized_img'
out_mask_dir = '/content/drive/MyDrive/resized_seg'

os.makedirs(out_img_dir, exist_ok=True)
os.makedirs(out_mask_dir, exist_ok=True)

for subfolder in os.listdir(main_folder):
    subfolder_path = os.path.join(main_folder, subfolder)
    img_path = os.path.join(subfolder_path, "imaging_middle.nii.gz")
    mask_path = os.path.join(subfolder_path, "segmentation_middle.nii.gz")
    
    if os.path.isfile(img_path) and os.path.isfile(mask_path):
        img_data = nib.load(img_path).get_fdata()
        img_data = np.squeeze(img_data) 
        img_data = cv2.resize(img_data, (SIZE_Y, SIZE_X))
        
        out_img_path = os.path.join(out_img_dir, subfolder + ".nii.gz")
        nib.save(nib.Nifti1Image(img_data, affine=None), out_img_path)

        mask_data = nib.load(mask_path).get_fdata()
        mask_data = np.squeeze(mask_data)  
        mask_data = cv2.resize(mask_data, (SIZE_Y, SIZE_X), interpolation=cv2.INTER_NEAREST)

        out_mask_path = os.path.join(out_mask_dir, subfolder + ".nii.gz")
        nib.save(nib.Nifti1Image(mask_data, affine=None), out_mask_path)


img_files = os.listdir(out_img_dir)
for img_file in img_files:
    img_path = os.path.join(out_img_dir, img_file)
    img_data = nib.load(img_path).get_fdata()
    print(f"Image: {img_file} | Dimensions: {img_data.shape}")
seg_files = os.listdir(out_mask_dir)
for seg_file in seg_files:
    seg_path = os.path.join(out_mask_dir, seg_file)
    seg_data = nib.load(seg_path).get_fdata()
    print(f"Segmentation: {seg_file} | Dimensions: {seg_data.shape}")

NameError: ignored

In [ ]:
#Function for resized images visualization

def display_resized_images(img_dir, mask_dir):
    img_files = os.listdir(img_dir)
    num_images = len(img_files)
    num_columns = 5
    num_rows = -(-num_images // num_columns)  # Round up division

    fig = plt.figure(figsize=(20, num_rows * 4))
    grid = ImageGrid(fig, 111, nrows_ncols=(num_rows, num_columns), axes_pad=0.5)

    for i, img_file in enumerate(img_files):
        img_path = os.path.join(img_dir, img_file)
        mask_path = os.path.join(mask_dir, img_file)
        img_data = nib.load(img_path).get_fdata()
        mask_data = nib.load(mask_path).get_fdata()

        ax = grid[i]
        ax.imshow(img_data, cmap='gray')
        ax.imshow(mask_data, cmap='jet', alpha=0.3)
        ax.set_title(img_file)
        ax.axis('off')

  
    plt.show()

display_resized_images(out_img_dir, out_mask_dir)

In [11]:
#loaded_img and loaded_seg lists to np.array

loaded_img = []
loaded_seg = []

for case_folder in os.listdir(out_img_dir):
    img_path = os.path.join(out_img_dir, case_folder)
    mask_path = os.path.join(out_mask_dir, case_folder)
    
    if not os.path.isfile(img_path) or not os.path.isfile(mask_path):
        continue  

    img = nib.load(img_path).get_fdata()
    mask = nib.load(mask_path).get_fdata()
    
    loaded_img.append(img)
    loaded_seg.append(mask)
loaded_img = np.array(loaded_img)
loaded_seg = np.array(loaded_seg)

print("Size Images:", loaded_img.shape)
print("Size Segmentations:", loaded_seg.shape)

Size Images: (101, 256, 256)
Size Segmentations: (101, 256, 256)


In [ ]:
# Checking the classes of the masks

unique_classes = np.unique(loaded_seg)

if len(unique_classes) == 3 and 0 in unique_classes and 1 in unique_classes and 2 in unique_classes:
    print("The masks are labeled with three classes: background, kidney, and tumor.")
else:
    print("The masks are not properly labeled with three classes.")

unique_values = np.unique(loaded_seg)
print("Unique values in the masks array:", unique_values)

The masks are labeled with three classes: background, kidney, and tumor.
Unique values in the masks array: [0. 1. 2.]


# Model

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [13]:
#@title Model: multi_unet
n_classes = 3
kernel_initializer =  'he_uniform'

################################################################
def multi_unet_model(n_classes=3, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=1):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255)(inputs) 
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l1_l2(l1=0.005, l2=0.01))(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(l2=0.005))(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
     
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

#model.summary()

# Splitting and One-hot Encoding

In [15]:
# Splitting into Validation Training and Test
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2
loaded_img = np.array(loaded_img)
loaded_seg = np.array(loaded_seg)

X_train_val, X_test, y_train_val, y_test = train_test_split(loaded_img, loaded_seg, test_size=test_ratio, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=val_ratio/(1 - test_ratio), random_state=42
)
print("Train set size:", X_train.shape[0])
print("Validation set size:", X_val.shape[0])
print("Test set size:", X_test.shape[0])



Train set size: 60
Validation set size: 20
Test set size: 21


In [18]:
X_train = np.expand_dims(X_train, axis=3)
X_val = np.expand_dims(X_val, axis=3)
X_test = np.expand_dims(X_test, axis=3)
y_train = np.expand_dims(y_train, axis=3)
y_val = np.expand_dims(y_val, axis=3)
y_test = np.expand_dims(y_test, axis=3)
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)

X_train shape: (60, 256, 256, 1)
X_val shape: (20, 256, 256, 1)
X_test shape: (21, 256, 256, 1)
y_train shape: (60, 256, 256, 1)
y_val shape: (20, 256, 256, 1)
y_test shape: (21, 256, 256, 1)


In [19]:
#One hot encoding on y_train
import numpy as np
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)
y_val = to_categorical(y_val, num_classes=3)


## Augmentation

In [21]:
def augment_data(X_train, y_train):
    datagen = ImageDataGenerator(
        rotation_range=30,  
        width_shift_range=0.2,  
        height_shift_range=0.2, 
        zoom_range=0.3,  
        horizontal_flip=True,  
        fill_mode='nearest'  
    )

    augmented_images = []
    augmented_masks = []

    for i in range(len(X_train)):
        image = X_train[i]
        mask = y_train[i]

        concatenated = np.concatenate((image, mask), axis=2)
        concatenated = concatenated.reshape((1,) + concatenated.shape)
        aug_iter = datagen.flow(concatenated, batch_size=1)
        augmented = next(aug_iter)
        augmented_image = augmented[..., :1]
        augmented_mask = augmented[..., 1:]
        augmented_images.append(augmented_image[0])
        augmented_masks.append(augmented_mask[0])
    augmented_images = np.array(augmented_images)
    augmented_masks = np.array(augmented_masks)

    return augmented_images, augmented_masks

In [ ]:
#@title Augmented Data Display
def display_augmented_data(X_train_augmented, y_train_augmented, num_samples=3):
    sample_indices = random.sample(range(len(X_train_augmented)), min(num_samples, 9))
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    fig.subplots_adjust(hspace=0.4)

    for i, idx in enumerate(sample_indices):
        row = i // 3
        col = i % 3

        image = X_train_augmented[idx]
        mask = y_train_augmented[idx]

        axes[row, col].imshow(image[..., 0], cmap='gray')
        axes[row, col].axis('off')
        axes[row, col].imshow(mask[..., 0], alpha=0.5, cmap='jet')

    for j in range(num_samples, 9):
        row = j // 3
        col = j % 3
        axes[row, col].axis('off')

    plt.show()

X_train_augmented, y_train_augmented = augment_data(X_train, y_train)
display_augmented_data(X_train_augmented, y_train_augmented, num_samples=9)

# Train Loading

In [23]:
def train_loading(X_train, y_train, X_train_augmented, y_train_augmented, num_augmented=20):
    indices = np.arange(len(X_train))
    np.random.shuffle(indices)
    original_indices = indices
    augmented_indices = np.random.choice(len(X_train_augmented), num_augmented)
    original_images = X_train[original_indices]
    original_masks = y_train[original_indices]
    augmented_images = X_train_augmented[augmented_indices]
    augmented_masks = y_train_augmented[augmented_indices]
    X_train_full = np.concatenate([original_images, augmented_images], axis=0)
    y_train_full = np.concatenate([original_masks, augmented_masks], axis=0)

    return X_train_full, y_train_full

In [ ]:
#Visualizing final training Dataset
def visualize_random_samples(images, masks, num_samples=5):
    sample_indices = random.sample(range(len(images)), num_samples)

    fig, axes = plt.subplots(num_samples, 2, figsize=(8, 8))
    for i, idx in enumerate(sample_indices):
        image = images[idx]
        mask = masks[idx]

        # Plot the image
        axes[i, 0].imshow(image, cmap='gray')
        axes[i, 0].axis('off')
        axes[i, 0].set_title('Image')

        # Plot the mask
        axes[i, 1].imshow(mask, cmap='gray')
        axes[i, 1].axis('off')
        axes[i, 1].set_title('Mask')

    plt.tight_layout()
    plt.show()

# Assuming you have loaded your dataset into X_train_full and y_train_full
visualize_random_samples(X_train_full, y_train_full, num_samples=5)

In [25]:
# Function for weighted losses
def weighted_loss(class_weights):
    def loss(y_true, y_pred):
        y_true = tf.argmax(y_true, axis=-1)
        class_weights_tensor = tf.convert_to_tensor(list(class_weights.values()), dtype=tf.float32)
        weights = tf.gather(class_weights_tensor, y_true)
        loss_value = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred, sample_weight=weights)
        
        return loss_value
    return loss

# Training and Evaluation

In [ ]:

n_classes = 3
IMG_HEIGHT = 256
IMG_WIDTH  = 256
IMG_CHANNELS = 1

checkpoint_filepath = '/content/drive/MyDrive/models/checkpoints/model_checkpoint.hdf5'

def get_model():
    model = multi_unet_model(n_classes=n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)
    return model

model = get_model()
model.compile(optimizer = Adam(learning_rate = 1e-3),loss=weighted_loss(class_weights_dict), metrics=['accuracy'])
#model.summary()
X_train_full, y_train_full = train_loading(X_train, y_train, X_train_augmented, y_train_augmented)

checkpoint_filepath = '/content/drive/MyDrive/models/checkpoints/model_checkpoint.hdf5'

checkpoint = ModelCheckpoint(checkpoint_filepath, 
                             monitor='loss', 
                             save_best_only=True, 
                             mode='min', 
                             verbose=1)
#for pre-trained weights. 
#model.load_weights('')

history = model.fit(X_train_full, y_train_full, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=110, 
                    validation_data=(X_val, y_val), 
                    #class_weight=class_weights,
                    shuffle=True,
                    callbacks=[checkpoint])
                   
model.save('test.hdf5')


In [ ]:

_, acc = model.evaluate(X_test, y_test)
print("Accuracy is = ", (acc * 100.0), "%")

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Testing

In [ ]:

_, acc = model.evaluate(X_test, y_test)
print("Accuracy is = ", (acc * 100.0), "%")

# Select a random image from the test dataset
index = np.random.randint(len(X_test))
test_image = X_test[index]
ground_truth = y_test[index]

test_image = np.reshape(test_image, (1,) + test_image.shape)
predicted_mask = model.predict(test_image)

predicted_labels = np.argmax(predicted_mask, axis=-1)
ground_truth_labels = np.argmax(ground_truth, axis=-1)

plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.imshow(test_image[0])
plt.title("Original Image")

plt.subplot(1, 3, 2)
plt.imshow(ground_truth_labels, cmap='gray')
plt.title("Ground Truth")

plt.subplot(1, 3, 3)
plt.imshow(predicted_labels[0], cmap='gray')
plt.title("Predicted Segmentation")

plt.tight_layout()
plt.show()